# Data Analysis for E-Commerce Challenge

# Import and Read Dataframe

In [1]:
#import Library
import pandas as pd

#Load Data
order_details = pd.read_csv('order_details.csv', sep=';')
orders = pd.read_csv('orders.csv', sep=';')
products = pd.read_csv('products.csv', sep=';')
users = pd.read_csv('users.csv', sep=';')

#Checking Data
print('order_details shape: ',order_details.shape)
print(order_details.isna().sum())
print('\n orders shape: ', orders.shape)
print(orders.isna().sum())
print('\nproducts shape: ', products.shape)
print(products.isna().sum())
print('\nusers shape: ', users.shape)
print(users.isna().sum())

order_details shape:  (187452, 5)
order_detail_id    0
order_id           0
product_id         0
price              0
quantity           0
dtype: int64

 orders shape:  (74874, 10)
order_id          0
seller_id         0
buyer_id          0
kodepos           0
subtotal          0
discount          0
total             0
created_at        0
paid_at        5046
delivery_at    9790
dtype: int64

products shape:  (1145, 4)
product_id      0
desc_product    0
category        0
base_price      0
dtype: int64

users shape:  (17936, 4)
user_id      0
nama_user    0
kodepos      0
email        0
dtype: int64


In [2]:
#Manipulating orders
df = orders.copy()
df['created_at'] = pd.to_datetime(df['created_at'])
df['Month_created_at'] = df['created_at'].dt.to_period('m')
Month = df.copy()
Month = df.groupby(['Month_created_at'])['order_id'].count()
print(Month)

Month_created_at
2019-01      117
2019-02      354
2019-03      668
2019-04      984
2019-05     1462
2019-06     1913
2019-07     2667
2019-08     3274
2019-09     4327
2019-10     5577
2019-11     7162
2019-12    10131
2020-01     5062
2020-02     5872
2020-03     7323
2020-04     7955
2020-05    10026
Freq: M, Name: order_id, dtype: int64


# Orders Sameday

In [40]:
#Manipulating orders to know delivery_at = paid_at
sameday = df.copy()
sameday['sameday'] = sameday['delivery_at'] == sameday['paid_at']
print(sameday[sameday['sameday']==True].shape)

(4588, 12)


Now we know that there were 4.588 orders delivered on the same day that the customer paid.

# Users perCategory

In [33]:
#Manipulating orders to determine how many users are buyer-only, seller-only, or both.
seller = set(df['seller_id'])
buyer = set(df['buyer_id'])
both = seller.intersection(buyer)
seller = pd.DataFrame(seller)
buyer = pd.DataFrame(buyer)
both = pd.DataFrame(both)
print('Number of all users: ', users['user_id'].count())
print('Number of sellers: ', seller.count())
print('Number of buyers: ', buyer.count())
print('Number of both: ', both.count())

Number of all users:  17936
Number of sellers:  0    69
dtype: int64
Number of buyers:  0    17877
dtype: int64
Number of both:  0    69
dtype: int64


In [61]:
print('Number of buyer-only: ', 17877-69)
print('Number of seller-only: ', 69 - 69)
print('Number of users without transaction: ', 17936 - 17877)

Number of buyer-only:  17808
Number of seller-only:  0
Number of users without transaction:  59


Now we know that from 17.936 users, there are 17.808 as buyer-only, 0 as seller only, 69 as both, and 59 with no transaction.

# Top 5 Users with Biggest Nominal of Transaction

In [7]:
#Determining the user with biggest nominal of transaction
biggest_total = df.copy()
biggest_total = biggest_total.drop(['order_id', 'seller_id', 'subtotal', 'discount', 'created_at', 'paid_at', 'delivery_at', 'Month_created_at'], axis=1)
top_5 = biggest_total.groupby(['buyer_id'])['total'].sum()
top_5 = top_5.sort_values(ascending=False)
top_5 = top_5.head()
print(top_5)

buyer_id
14411    54102250
11140    52743200
15915    49141800
2908     49033000
10355    48868000
Name: total, dtype: int64


In [8]:
user = users.copy()
biggest_total_top5 = ['14411', '11140', '15915', '2908', '10355']
top_5_big = user[(user['user_id'].isin(biggest_total_top5))]
print(top_5_big)

       user_id                      nama_user  kodepos  \
2907      2908            Septi Melani, S.Ked    68589   
10354    10355                 Kartika Habibi      869   
11139    11140  R.A. Yulia Padmasari, S.I.Kom    65882   
14410    14411                 Jaga Puspasari    24303   
15914    15915     Sutan Agus Ardianto, S.Kom    42909   

                           email  
2907      yuniaraurora@gmail.com  
10354        siregarbakti@pd.gov  
11139     safitriunggul@pt.co.id  
14410  pratamacawisadi@yahoo.com  
15914       twidiastuti@pt.go.id  


Top Buyer All Time
    1. Jaga Puspasari                      14411   Rp. 54.102.250
    2. R.A. Yulia Padmasari, S.I.Kom       11140   Rp. 52.743.200
    3. Sutan Agus Ardianto, S.Kom          15915   Rp. 49.141.800
    4. Septi Melani, S.Ked                  2908   Rp. 49.033.000
    5. Kartika Habibi                      10355   Rp. 48.868.000

# Most Frequent Buyer (without discount)

In [69]:
#Manipulating orders to know buyer with most frequence order without discount
no_disc = df.copy()
no_disc['with_discount'] = no_disc['discount'] != 0
true = [True]
no_disc = no_disc[no_disc.with_discount.isin(true) == False]
no_disc = no_disc.groupby(['buyer_id'])['order_id'].count()
no_disc = no_disc.sort_values(ascending = False)
print(no_disc.head(10))

buyer_id
12476    13
10977    12
12577    12
14309    11
1251     11
7543     11
5620     11
14172    11
9260     11
14095    11
Name: order_id, dtype: int64


In [70]:
no_disc_top5 = ['12476', '10977', '12577', '14309', '1251', '7543', '5620', '14172', '9260', '14095']
top_5_freq = users[(users['user_id'].isin(no_disc_top5))]
print(top_5_freq)

       user_id                 nama_user  kodepos                      email
1250      1251              Septi Sinaga    61651      namagakayla@gmail.com
5619      5620        Cakrawangsa Habibi     4010   namagaargono@hotmail.com
7542      7543           Laras Puspasari    24163         rahmat51@pd.mil.id
9259      9260           Bakiono Zulaika    33416    candrakanta88@gmail.com
10976    10977  Drs. Pandu Mansur, M.TI.    16333          narji63@yahoo.com
12475    12476            Yessi Wibisono    16556    parmanwaskita@gmail.com
12576    12577             Umay Latupono    63191        lpradipta@perum.com
14094    14095             Kamal Fujiati     3045  yuliantiharsaya@ud.biz.id
14171    14172   Ophelia Oktaviani, S.H.    73282       mutiawidodo@ud.go.id
14308    14309    Diah Nainggolan, S.E.I    92892    idaadriansyah@pt.biz.id


Most Frequent Buyers without discount
    1. 12476    Yessi Wibisono             13x
    2. 12577    Umay Latupono              12x
    3. 10977    Drs. Pandu Mansur, M.TI.   12x
    4. 14309    Diah Nainggolan, S.E.I     11x
    5. 1251     Septi Sinaga               11x
    6. 7543     Laras Puspasari            11x
    7. 5620     Cakrawangsa Habibi         11x
    8. 14172    Ophelia Oktaviani, S.H.    11x
    9. 9260     Bakiono Zulaika            11x
    10.14095    Kamal Fujiati              11x

In [28]:
#Big Frequent Buyer 2020
quest = ['12854', '11195', '10028', '3185', '11998']
big_freq_buy = big_freq[(big_freq['buyer_id'].isin(quest))]
big_freq_buy = big_freq_buy.sort_values(by='created_at', ascending = True)
big_freq_buy = big_freq_buy.sort_values(by='buyer_id', ascending = False)
big_freq_buy = big_freq_buy.drop(['order_id', 'seller_id', 'kodepos', 'subtotal', 'discount', 'paid_at', 'delivery_at', 'Month_created_at', 'twenty'], axis=1)
print(big_freq_buy)

       buyer_id     total created_at
60188     12854  29143000 2020-05-04
74489     11998   3328400 2020-01-01
39904     11998   4087000 2020-05-21
60768     11998   5978000 2020-02-25
36874     11998   2126000 2020-03-29
2792      11998   8985200 2020-04-28
55158     11195  17156000 2020-04-29
57545     11195   4062000 2020-05-20
58740     11195   2831000 2020-02-19
65316     11195   6637000 2020-03-26
53844     11195    114000 2020-01-03
32481     11195   4405000 2020-04-25
18177     10028  27136000 2020-05-27
64040      3185    168000 2020-03-09
45130      3185  13500000 2020-02-18
42667      3185    663000 2020-01-25
15050      3185   2587400 2020-05-26
64777      3185   9634000 2020-04-02


From that output, we know that buyer_id 11998, 11195, and 3185 have an order atleast once every month on 2020 (untill May).

# Email Domain of The Sellers

In [32]:
#Determine email domain of the seller
seller = df['seller_id']
Seller = users[users.user_id.isin(seller) == True]
Seller['domain'] = Seller['email'].apply(lambda x: x.split('@')[1])
Seller_domain = Seller.groupby(['domain'])['user_id'].count()
print(Seller_domain)

domain
cv.co.id         1
cv.id            1
cv.mil           2
cv.web.id        2
gmail.com       16
hotmail.com     14
pd.ac.id         1
pd.go.id         3
pd.mil.id        1
pd.my.id         1
pd.net           1
pd.org           1
pd.sch.id        1
pd.web.id        1
perum.edu        2
perum.int        1
perum.mil        1
perum.mil.id     1
pt.gov           1
pt.mil.id        1
pt.net.id        3
ud.edu           2
ud.go.id         1
ud.id            2
ud.net           1
ud.net.id        2
yahoo.com        5
Name: user_id, dtype: int64


<ipython-input-32-23360e44eb59>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Seller['domain'] = Seller['email'].apply(lambda x: x.split('@')[1])


# Top 5 Product on December 2019

In [16]:
#Determining Top 5 Product on December 2019
#Take order_id and month_created_at form orders to know the time of transaction
time_order = df[['order_id', 'Month_created_at']].copy()
print(time_order.head())
print(time_order.shape)
#Take order_id, order_detail_id and product_id from order_details to know the product on each order
product_order = order_details[['order_id', 'order_detail_id', 'product_id', 'quantity']].copy()
print(product_order.head())
print(product_order.shape)
#Take product_id and desc_product from products to know the product name of the product_id
product_name = products[['product_id', 'desc_product']].copy()
print(product_name.head())
print(product_name.shape)
#print(df.head())
#print(order_details.head())

   order_id Month_created_at
0         3          2019-06
1         5          2019-04
2         8          2019-08
3        19          2020-05
4        31          2019-03
(74874, 2)
   order_id  order_detail_id  product_id  quantity
0         3                5         907        36
1         5                9         562        22
2         8               15         645        32
3        19               48         287        12
4        19               49         201        27
(187452, 4)
   product_id                desc_product
0           1         OLIVIA KULOT OLV03 
1           2        BLANIK BLOUSE BL304 
2           3  NEW DAY BY RIX DRESS ND01 
3           4        BLANIK BLOUSE BL023 
4           5        BLANIK BLAZER BL031 
(1145, 2)


In [32]:
merged = product_order.merge(time_order,on='order_id').merge(product_name,on='product_id')
merged = merged.sort_values(by='order_id', ascending = True)
merged['Dec_2019'] = merged['Month_created_at'] > '2019-12'
true = [True]
merged = merged[merged.Dec_2019.isin(true) == False]
merged['Dec_2019'] = merged['Month_created_at'] < '2019-12'
merged = merged[merged.Dec_2019.isin(true) == False]
group_merged = merged.groupby(['desc_product'])['quantity'].sum()
group_merged = group_merged.sort_values(ascending = False)
print(group_merged.head(5))

desc_product
QUEEN CEFA BRACELET LEATHER                         2550
SHEW SKIRTS BREE                                    1423
ANNA FAITH LEGGING GLOSSY                           1323
Cdr Vitamin C 10'S                                  1242
RIDER CELANA DEWASA SPANDEX ANTI BAKTERI R325BW     1186
Name: quantity, dtype: int64
